In [1]:
# PEP8 Standard: All libraries utilized in this notebook will be imported in this cell 
import pandas as pd

# About the DOB Permit Data

Source: https://data.cityofnewyork.us/Housing-Development/DOB-Permit-Issuance/ipu4-2q9a

Introduction:
- The Department of Buildings (DOB) issues permits for construction and demolition activities in the City of New York. The construction industry must submit an application to DOB with details of the construction job they would like to complete.


Technicalities:
- Dataset is updated daily with new records. Moving forward, if it is decided that the permits are an important part of the model, will need to write a script that pulls and update data record daily, automatically. 


# Reading and understanding the data

In [2]:
!curl https://data.cityofnewyork.us/api/views/ipu4-2q9a/rows.csv?accessType=DOWNLOAD > DOB_Permit_Issuance.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1582M    0 1582M    0     0  5535k      0 --:--:--  0:04:52 --:--:-- 4803k


In [3]:
permit = pd.read_csv("DOB_Permit_Issuance.csv")

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,8,9,10,15,25,31,33,34,35,36,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
permit.shape

(3564341, 60)

In [5]:
#showing only the headers
permit.columns

Index(['BOROUGH', 'Bin #', 'House #', 'Street Name', 'Job #', 'Job doc. #',
       'Job Type', 'Self_Cert', 'Block', 'Lot', 'Community Board', 'Zip Code',
       'Bldg Type', 'Residential', 'Special District 1', 'Special District 2',
       'Work Type', 'Permit Status', 'Filing Status', 'Permit Type',
       'Permit Sequence #', 'Permit Subtype', 'Oil Gas', 'Site Fill',
       'Filing Date', 'Issuance Date', 'Expiration Date', 'Job Start Date',
       'Permittee's First Name', 'Permittee's Last Name',
       'Permittee's Business Name', 'Permittee's Phone #',
       'Permittee's License Type', 'Permittee's License #',
       'Act as Superintendent', 'Permittee's Other Title', 'HIC License',
       'Site Safety Mgr's First Name', 'Site Safety Mgr's Last Name',
       'Site Safety Mgr Business Name', 'Superintendent First & Last Name',
       'Superintendent Business Name', 'Owner's Business Type', 'Non-Profit',
       'Owner's Business Name', 'Owner's First Name', 'Owner's Last Name',
 

# Narrow down to columns of interest

Spatial columns: Borough, census tract, NTA name, Zip Code, Block, Lot, Latitude, Longtitude

Time columns: Issuance Date, Expiration Date, Job Start Date

Attributes: 
- Job Type
- Work Type
- Permit Type
- Permit Subtype


In [6]:
permit2 = permit[['BOROUGH', 'CENSUS_TRACT', 'NTA_NAME', 'Zip Code', 'Block', 'Lot', 'LATITUDE', 'LONGITUDE',
              'Job Type', 'Work Type', 'Permit Type', 'Permit Subtype', 'Oil Gas', 'Site Fill',
              'Issuance Date', 'Expiration Date', 'Job Start Date']]

In [7]:
permit2.head()

,BOROUGH,CENSUS_TRACT,NTA_NAME,Zip Code,Block,Lot,LATITUDE,LONGITUDE,Job Type,Work Type,Permit Type,Permit Subtype,Oil Gas,Site Fill,Issuance Date,Expiration Date,Job Start Date
0,BRONX,310.0,Allerton-Pelham Gardens,10469.0,4416,1,40.859247,-73.833406,A2,OT,EW,OT,NaN,NOT APPLICABLE,07/21/2019 12:00:00 AM,08/05/2020 12:00:00 AM,04/09/2019 12:00:00 AM
1,BROOKLYN,159.0,Park Slope-Gowanus,11217.0,948,13,40.676927,-73.975351,A2,OT,EW,OT,NaN,NOT APPLICABLE,07/21/2019 12:00:00 AM,06/12/2020 12:00:00 AM,07/21/2019 12:00:00 AM
2,MANHATTAN,111.0,Hudson Yards-Chelsea-Flatiron-Union Square,10018.0,760,63,40.754573,-73.993240,A2,PL,PL,NaN,NaN,NOT APPLICABLE,07/21/2019 12:00:00 AM,07/20/2020 12:00:00 AM,07/21/2019 12:00:00 AM
3,STATEN ISLAND,64.0,Grasmere-Arrochar-Ft. Wadsworth,10305.0,3269,65,40.597142,-74.074614,A1,NaN,AL,NaN,NaN,NOT APPLICABLE,07/21/2019 12:00:00 AM,08/05/2020 12:00:00 AM,04/22/2019 12:00:00 AM
4,BROOKLYN,571.0,Greenpoint,11222.0,2685,14,40.724536,-73.945281,A2,OT,EW,OT,NaN,NOT APPLICABLE,07/21/2019 12:00:00 AM,06/12/2020 12:00:00 AM,07/21/2019 12:00:00 AM


In [8]:
permit2.dtypes

BOROUGH             object
CENSUS_TRACT       float64
NTA_NAME            object
Zip Code           float64
Block               object
Lot                 object
LATITUDE           float64
LONGITUDE          float64
Job Type            object
Work Type           object
Permit Type         object
Permit Subtype      object
Oil Gas             object
Site Fill           object
Issuance Date       object
Expiration Date     object
Job Start Date      object
dtype: object

# TIME

### Changing the columns of interest into datetime format
Removing the timestamp

In [9]:
# code credit @https://stackoverflow.com/questions/32888124/pandas-out-of-bounds-nanosecond-timestamp-after-offset-rollforward-plus-adding-a

permit2['JobStartDate'] = pd.to_datetime(permit2['Job Start Date'], errors = 'coerce')
permit2['ExpireDate'] = pd.to_datetime(permit2['Expiration Date'], errors = 'coerce')
permit2['IssueDate'] = pd.to_datetime(permit2['Issuance Date'], errors = 'coerce')

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [10]:
#extracting just the year information
permit2['JobYear'] = permit2['JobStartDate'].apply(lambda x: x.year)
permit2['ExpireYear'] = permit2['ExpireDate'].apply(lambda x: x.year)
permit2['IssueYear'] = permit2['IssueDate'].apply(lambda x: x.year)

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to b

In [11]:
permit2.dtypes

BOROUGH                    object
CENSUS_TRACT              float64
NTA_NAME                   object
Zip Code                  float64
Block                      object
Lot                        object
LATITUDE                  float64
LONGITUDE                 float64
Job Type                   object
Work Type                  object
Permit Type                object
Permit Subtype             object
Oil Gas                    object
Site Fill                  object
Issuance Date              object
Expiration Date            object
Job Start Date             object
JobStartDate       datetime64[ns]
ExpireDate         datetime64[ns]
IssueDate          datetime64[ns]
JobYear                   float64
ExpireYear                float64
IssueYear                 float64
dtype: object

In [12]:
permit2.head()

,BOROUGH,CENSUS_TRACT,NTA_NAME,Zip Code,Block,Lot,LATITUDE,LONGITUDE,Job Type,Work Type,...,Site Fill,Issuance Date,Expiration Date,Job Start Date,JobStartDate,ExpireDate,IssueDate,JobYear,ExpireYear,IssueYear
0,BRONX,310.0,Allerton-Pelham Gardens,10469.0,4416,1,40.859247,-73.833406,A2,OT,...,NOT APPLICABLE,07/21/2019 12:00:00 AM,08/05/2020 12:00:00 AM,04/09/2019 12:00:00 AM,2019-04-09,2020-08-05,2019-07-21,2019.0,2020.0,2019.0
1,BROOKLYN,159.0,Park Slope-Gowanus,11217.0,948,13,40.676927,-73.975351,A2,OT,...,NOT APPLICABLE,07/21/2019 12:00:00 AM,06/12/2020 12:00:00 AM,07/21/2019 12:00:00 AM,2019-07-21,2020-06-12,2019-07-21,2019.0,2020.0,2019.0
2,MANHATTAN,111.0,Hudson Yards-Chelsea-Flatiron-Union Square,10018.0,760,63,40.754573,-73.993240,A2,PL,...,NOT APPLICABLE,07/21/2019 12:00:00 AM,07/20/2020 12:00:00 AM,07/21/2019 12:00:00 AM,2019-07-21,2020-07-20,2019-07-21,2019.0,2020.0,2019.0
3,STATEN ISLAND,64.0,Grasmere-Arrochar-Ft. Wadsworth,10305.0,3269,65,40.597142,-74.074614,A1,NaN,...,NOT APPLICABLE,07/21/2019 12:00:00 AM,08/05/2020 12:00:00 AM,04/22/2019 12:00:00 AM,2019-04-22,2020-08-05,2019-07-21,2019.0,2020.0,2019.0
4,BROOKLYN,571.0,Greenpoint,11222.0,2685,14,40.724536,-73.945281,A2,OT,...,NOT APPLICABLE,07/21/2019 12:00:00 AM,06/12/2020 12:00:00 AM,07/21/2019 12:00:00 AM,2019-07-21,2020-06-12,2019-07-21,2019.0,2020.0,2019.0


Noticed that the date of the permit can be after the job started! Makes sense that permit might only be applied after job started- to add on new construction projects. 

In [13]:
permit2a = permit2

### Getting some descriptive stats of Issue, Expire and Job Start Date...

In [14]:
#coding hack: when you sum boolean, True=1 and False=0

nanStart = permit2a['JobStartDate'].isnull()
print ('No. of data records that have NaN for Job start date')
print (nanStart.sum())

nanIssue = permit2a['IssueDate'].isnull()
print ('No. of data records that have NaN for Issue date')
print (nanIssue.sum())

nanExpire = permit2a['ExpireDate'].isnull()
print ('No. of data records that have NaN for Expire date')
print (nanExpire.sum())

nanAll = permit2a['IssueDate'].isnull() & permit2a['JobStartDate'].isnull() & permit2a['ExpireDate'].isnull()
print ('No. of data records that have NaN for issue, expire and job start date')
print (nanAll.sum()) #Returns only the sum of True values

nanIssueExpire = permit2a['IssueDate'].isnull() & permit2a['ExpireDate'].isnull()
print ('No. of data records that have NaN issue and expire date') 
print (nanIssueExpire.sum()) 

nanIssueJob = permit2a['IssueDate'].isnull() & permit2a['JobStartDate'].isnull()
print ('No. of data records that have NaN issue and job Start date') 
print (nanIssueJob.sum()) 

nanExpireJob = permit2a['ExpireDate'].isnull() & permit2a['JobStartDate'].isnull()
print ('No. of data records that have NaN job start and expire date') 
print (nanExpireJob.sum()) 


No. of data records that have NaN for Job start date
40
No. of data records that have NaN for Issue date
20295
No. of data records that have NaN for Expire date
11456
No. of data records that have NaN for issue, expire and job start date
1
No. of data records that have NaN issue and expire date
11452
No. of data records that have NaN issue and job Start date
2
No. of data records that have NaN job start and expire date
1


In [15]:
ExpireJobWerid = permit2a['ExpireYear'] < permit2a['JobYear']
print ('No. of data records that have job starting after permit expired')
print (ExpireJobWerid.sum())

No. of data records that have job starting after permit expired
1500


In [16]:
werid = permit2a[permit2a['ExpireYear'] < permit2a['JobYear']]
werid.head()

,BOROUGH,CENSUS_TRACT,NTA_NAME,Zip Code,Block,Lot,LATITUDE,LONGITUDE,Job Type,Work Type,...,Site Fill,Issuance Date,Expiration Date,Job Start Date,JobStartDate,ExpireDate,IssueDate,JobYear,ExpireYear,IssueYear
311,MANHATTAN,113.0,Midtown-Midtown South,10018.0,789,62,40.755265,-73.989132,A2,OT,...,NOT APPLICABLE,07/19/2019 12:00:00 AM,02/26/2020 12:00:00 AM,08/09/2108 12:00:00 AM,2108-08-09,2020-02-26,2019-07-19,2108.0,2020.0,2019.0
2463,BROOKLYN,383.0,Stuyvesant Heights,11221.0,1657,38,40.685863,-73.927558,A3,OT,...,NaN,07/16/2019 12:00:00 AM,09/13/2019 12:00:00 AM,07/16/2029 12:00:00 AM,2029-07-16,2019-09-13,2019-07-16,2029.0,2019.0,2019.0
5236,QUEENS,947.0,College Point,11356.0,4095,8,40.784548,-73.840327,A2,PL,...,NOT APPLICABLE,07/10/2019 12:00:00 AM,07/09/2020 12:00:00 AM,03/26/2029 12:00:00 AM,2029-03-26,2020-07-09,2019-07-10,2029.0,2020.0,2019.0
9026,QUEENS,402.0,St. Albans,11412.0,10438,22,40.701778,-73.765495,A2,OT,...,NOT APPLICABLE,06/28/2019 12:00:00 AM,11/17/2019 12:00:00 AM,06/28/2049 12:00:00 AM,2049-06-28,2019-11-17,2019-06-28,2049.0,2019.0,2019.0
10104,STATEN ISLAND,27702.0,Todt Hill-Emerson Hill-Heartland Village-Light...,10314.0,2400,180,40.582305,-74.169053,A2,SP,...,NOT APPLICABLE,06/27/2019 12:00:00 AM,06/26/2020 12:00:00 AM,06/27/2109 12:00:00 AM,2109-06-27,2020-06-26,2019-06-27,2109.0,2020.0,2019.0


## Dropping Job start date after expire permits

In [17]:
#dropping rows of data that fulfills a boolean condition
#code credit@ https://stackoverflow.com/questions/13851535/how-to-delete-rows-from-a-pandas-dataframe-based-on-a-conditional-expression

permit3 = permit2a.drop(permit2a[permit2a['ExpireYear'] < permit2a['JobYear']].index)
len(permit2a) - len(permit3)

1500

In [18]:
len(permit3)

3562841

In [19]:
nanStart = permit3['JobYear'].isnull()
print ('No. of data records that have NaN for Job start date')
print (nanStart.sum())

nanIssue = permit3['IssueYear'].isnull()
print ('No. of data records that have NaN for Issue date')
print (nanIssue.sum())

nanExpire = permit3['ExpireYear'].isnull()
print ('No. of data records that have NaN for Expire date')
print (nanExpire.sum())

nanAll = permit3['IssueYear'].isnull() & permit3['JobYear'].isnull() & permit3['ExpireYear'].isnull()
print ('No. of data records that have NaN for issue, expire and job start date')
print (nanAll.sum()) #Returns only the sum of True values

nanIssueExpire = permit3['IssueYear'].isnull() & permit3['ExpireYear'].isnull()
print ('No. of data records that have NaN issue and expire date') 
print (nanIssueExpire.sum()) 

nanIssueJob = permit3['IssueYear'].isnull() & permit3['JobYear'].isnull()
print ('No. of data records that have NaN issue and job Start date') 
print (nanIssueJob.sum()) 

nanExpireJob = permit3['ExpireYear'].isnull() & permit3['JobYear'].isnull()
print ('No. of data records that have NaN job start and expire date') 
print (nanExpireJob.sum()) 

No. of data records that have NaN for Job start date
40
No. of data records that have NaN for Issue date
20278
No. of data records that have NaN for Expire date
11456
No. of data records that have NaN for issue, expire and job start date
1
No. of data records that have NaN issue and expire date
11452
No. of data records that have NaN issue and job Start date
2
No. of data records that have NaN job start and expire date
1


### Dropping that one data record that have nan for issue, expire and job start date

In [20]:
permit3a = permit3.drop(permit3[permit3['IssueYear'].isnull() &\
        permit3['JobYear'].isnull() & permit3['ExpireYear'].isnull()].index)

len(permit3) - len(permit3a)

1

In [21]:
permit3a.head()

,BOROUGH,CENSUS_TRACT,NTA_NAME,Zip Code,Block,Lot,LATITUDE,LONGITUDE,Job Type,Work Type,...,Site Fill,Issuance Date,Expiration Date,Job Start Date,JobStartDate,ExpireDate,IssueDate,JobYear,ExpireYear,IssueYear
0,BRONX,310.0,Allerton-Pelham Gardens,10469.0,4416,1,40.859247,-73.833406,A2,OT,...,NOT APPLICABLE,07/21/2019 12:00:00 AM,08/05/2020 12:00:00 AM,04/09/2019 12:00:00 AM,2019-04-09,2020-08-05,2019-07-21,2019.0,2020.0,2019.0
1,BROOKLYN,159.0,Park Slope-Gowanus,11217.0,948,13,40.676927,-73.975351,A2,OT,...,NOT APPLICABLE,07/21/2019 12:00:00 AM,06/12/2020 12:00:00 AM,07/21/2019 12:00:00 AM,2019-07-21,2020-06-12,2019-07-21,2019.0,2020.0,2019.0
2,MANHATTAN,111.0,Hudson Yards-Chelsea-Flatiron-Union Square,10018.0,760,63,40.754573,-73.993240,A2,PL,...,NOT APPLICABLE,07/21/2019 12:00:00 AM,07/20/2020 12:00:00 AM,07/21/2019 12:00:00 AM,2019-07-21,2020-07-20,2019-07-21,2019.0,2020.0,2019.0
3,STATEN ISLAND,64.0,Grasmere-Arrochar-Ft. Wadsworth,10305.0,3269,65,40.597142,-74.074614,A1,NaN,...,NOT APPLICABLE,07/21/2019 12:00:00 AM,08/05/2020 12:00:00 AM,04/22/2019 12:00:00 AM,2019-04-22,2020-08-05,2019-07-21,2019.0,2020.0,2019.0
4,BROOKLYN,571.0,Greenpoint,11222.0,2685,14,40.724536,-73.945281,A2,OT,...,NOT APPLICABLE,07/21/2019 12:00:00 AM,06/12/2020 12:00:00 AM,07/21/2019 12:00:00 AM,2019-07-21,2020-06-12,2019-07-21,2019.0,2020.0,2019.0


In [22]:
len(permit3a)

3562840

In [23]:
nanStart = permit3a['JobYear'].isnull()
print ('No. of data records that have NaN for Job start date')
print (nanStart.sum())

nanIssue = permit3a['IssueYear'].isnull()
print ('No. of data records that have NaN for Issue date')
print (nanIssue.sum())

nanExpire = permit3a['ExpireYear'].isnull()
print ('No. of data records that have NaN for Expire date')
print (nanExpire.sum())

nanAll = permit3a['IssueYear'].isnull() & permit3a['JobYear'].isnull() & permit3a['ExpireYear'].isnull()
print ('No. of data records that have NaN for issue, expire and job start date')
print (nanAll.sum()) #Returns only the sum of True values

nanIssueExpire = permit3a['IssueYear'].isnull() & permit3a['ExpireYear'].isnull()
print ('No. of data records that have NaN issue and expire date') 
print (nanIssueExpire.sum()) 

nanIssueJob = permit3a['IssueYear'].isnull() & permit3a['JobYear'].isnull()
print ('No. of data records that have NaN issue and job Start date') 
print (nanIssueJob.sum()) 

nanExpireJob = permit3a['ExpireYear'].isnull() & permit3a['JobYear'].isnull()
print ('No. of data records that have NaN job start and expire date') 
print (nanExpireJob.sum()) 

No. of data records that have NaN for Job start date
39
No. of data records that have NaN for Issue date
20277
No. of data records that have NaN for Expire date
11455
No. of data records that have NaN for issue, expire and job start date
0
No. of data records that have NaN issue and expire date
11451
No. of data records that have NaN issue and job Start date
1
No. of data records that have NaN job start and expire date
0


### Reducing to expire year that is from 2018

In [24]:
permit3a['ExpireYear'].unique()

array([ 2020.,  2019.,  2018.,  2017.,  2016.,  2027.,  2015.,  2014.,
        2013.,  2012.,  2011.,  2010.,  2009.,    nan,  2008.,  2209.,
        2097.,  2007.,  2006.,  2030.,  2005.,  2004.,  2003.,  2204.,
        2002.,  2202.,  2033.,  2001.,  2000.,  1999.,  2200.,  1998.,
        1997.,  1996.,  1995.,  1994.,  1993.,  1992.,  1991.,  1990.,
        1989.])

Checking that all the numbers add up first

In [25]:
ExpireFrom2018 = permit3a['ExpireYear'] > 2017
ExpireFrom2018.sum()

417878

In [26]:
#reminder to self: if we just choose < 2017, it will not include 2017!! 
ExpireBefore2018 = permit3a['ExpireYear'] <= 2017
ExpireBefore2018.sum()

3133507

In [27]:
ExpireNan = permit3a['ExpireYear'].isnull()
ExpireNan.sum()

11455

In [28]:
len(permit3a) - ExpireNan.sum() - ExpireBefore2018.sum() - ExpireFrom2018.sum()

0

Great, now all the numbers add up. And therefore should expect 361599 for next table cut

In [29]:
ExpireNan.sum() + ExpireFrom2018.sum()

429333

In [30]:
permit3b = permit3a.drop(permit3a[permit3a['ExpireYear'] <= 2017].index)
len(permit3b)

429333

Descriptive stats for permits3b

In [31]:
nanStart = permit3b['JobYear'].isnull()
print ('No. of data records that have NaN for Job start date')
print (nanStart.sum())

nanIssue = permit3b['IssueYear'].isnull()
print ('No. of data records that have NaN for Issue date')
print (nanIssue.sum())

nanExpire = permit3b['ExpireYear'].isnull()
print ('No. of data records that have NaN for Expire date')
print (nanExpire.sum())

nanAll = permit3b['IssueYear'].isnull() & permit3b['JobYear'].isnull() & permit3b['ExpireYear'].isnull()
print ('No. of data records that have NaN for issue, expire and job start date')
print (nanAll.sum()) #Returns only the sum of True values

nanIssueExpire = permit3b['IssueYear'].isnull() & permit3b['ExpireYear'].isnull()
print ('No. of data records that have NaN issue and expire date') 
print (nanIssueExpire.sum()) 

nanIssueJob = permit3b['IssueYear'].isnull() & permit3b['JobYear'].isnull()
print ('No. of data records that have NaN issue and job Start date') 
print (nanIssueJob.sum()) 

nanExpireJob = permit3b['ExpireYear'].isnull() & permit3b['JobYear'].isnull()
print ('No. of data records that have NaN job start and expire date') 
print (nanExpireJob.sum()) 

No. of data records that have NaN for Job start date
3
No. of data records that have NaN for Issue date
12908
No. of data records that have NaN for Expire date
11455
No. of data records that have NaN for issue, expire and job start date
0
No. of data records that have NaN issue and expire date
11451
No. of data records that have NaN issue and job Start date
0
No. of data records that have NaN job start and expire date
0


### Right now, the data has been cleaned to reflect all permits that expires either in 2018 or after 2018, as well as data records without Expire date entry

After analysizing the descriptive stats for permit3b data, the next step is to filter according to job start date

In [32]:
test = permit3b[permit3b['ExpireYear'].isnull()]
test.head()

,BOROUGH,CENSUS_TRACT,NTA_NAME,Zip Code,Block,Lot,LATITUDE,LONGITUDE,Job Type,Work Type,...,Site Fill,Issuance Date,Expiration Date,Job Start Date,JobStartDate,ExpireDate,IssueDate,JobYear,ExpireYear,IssueYear
1634105,QUEENS,462.0,Jamaica,11432.0,9834,33,40.711844,-73.787380,NB,NaN,...,ON-SITE,09/02/2009 12:00:00 AM,NaN,04/04/2008 12:00:00 AM,2008-04-04,NaT,2009-09-02,2008.0,NaN,2009.0
1664734,BROOKLYN,78.0,Sunset Park West,11220.0,773,51,40.647552,-74.012843,NB,NaN,...,OFF-SITE,06/15/2009 12:00:00 AM,NaN,01/16/2009 12:00:00 AM,2009-01-16,NaT,2009-06-15,2009.0,NaN,2009.0
1684537,MANHATTAN,47.0,SoHo-TriBeCa-Civic Center-Little Italy,10012.0,490,27,40.724963,-74.003633,NB,NaN,...,OFF-SITE,04/24/2009 12:00:00 AM,NaN,04/24/2009 12:00:00 AM,2009-04-24,NaT,2009-04-24,2009.0,NaN,2009.0
3179699,BROOKLYN,197.0,Clinton Hill,11238.0,1947,1,40.688353,-73.964511,A2,OT,...,NONE,11/07/1996 12:00:00 AM,NaN,11/07/1996 12:00:00 AM,1996-11-07,NaT,1996-11-07,1996.0,NaN,1996.0
3544046,QUEENS,294.0,Springfield Gardens North,11434.0,12258,00045,40.673420,-73.784463,NB,NaN,...,ON-SITE,NaN,NaN,10/22/2009 12:00:00 AM,2009-10-22,NaT,NaT,2009.0,NaN,NaN


In [33]:
test1 = permit3b[permit3b['JobYear'].isnull()]
test1

,BOROUGH,CENSUS_TRACT,NTA_NAME,Zip Code,Block,Lot,LATITUDE,LONGITUDE,Job Type,Work Type,...,Site Fill,Issuance Date,Expiration Date,Job Start Date,JobStartDate,ExpireDate,IssueDate,JobYear,ExpireYear,IssueYear
37416,MANHATTAN,6.0,Lower East Side,10002.0,284,19,40.714143,-73.989128,NB,NaN,...,OFF-SITE,05/01/2019 12:00:00 AM,04/30/2020 12:00:00 AM,NaN,NaT,2020-04-30,2019-05-01,NaN,2020.0,2019.0
258696,MANHATTAN,6.0,Lower East Side,10002.0,284,19,40.714143,-73.989128,NB,NaN,...,OFF-SITE,01/12/2018 12:00:00 AM,02/23/2018 12:00:00 AM,NaN,NaT,2018-02-23,2018-01-12,NaN,2018.0,2018.0
414988,MANHATTAN,6.0,Lower East Side,10002.0,284,19,40.714143,-73.989128,NB,NaN,...,OFF-SITE,03/28/2017 12:00:00 AM,02/23/2018 12:00:00 AM,NaN,NaT,2018-02-23,2017-03-28,NaN,2018.0,2017.0


Will not remove the data rows where job year is NaN because it falls within the 2018 time period based on the expire and issue date

In [34]:
ExpireNanJob2017 = permit3b['JobYear']<2018
ExpireNanJob2017.sum()

233165

In [35]:
ExpireNanJob2018 = permit3b['JobYear']>2017
ExpireNanJob2018.sum()

196165

In [36]:
len(permit3b) - ExpireNanJob2017.sum() - ExpireNanJob2018.sum()

3

Therefore should expect the next table cut to yield 

In [37]:
ExpireNanJob2018.sum() + len(test1)

196168

In [38]:
permit3c = permit3b.drop(permit3b[permit3b['JobYear'] <= 2017].index)
len(permit3c)

196168

In [39]:
nanStart = permit3c['JobYear'].isnull()
print ('No. of data records that have NaN for Job start date')
print (nanStart.sum())

nanIssue = permit3c['IssueYear'].isnull()
print ('No. of data records that have NaN for Issue date')
print (nanIssue.sum())

nanExpire = permit3c['ExpireYear'].isnull()
print ('No. of data records that have NaN for Expire date')
print (nanExpire.sum())

nanAll = permit3c['IssueYear'].isnull() & permit3c['JobYear'].isnull() & permit3c['ExpireYear'].isnull()
print ('No. of data records that have NaN for issue, expire and job start date')
print (nanAll.sum()) #Returns only the sum of True values

nanIssueExpire = permit3c['IssueYear'].isnull() & permit3c['ExpireYear'].isnull()
print ('No. of data records that have NaN issue and expire date') 
print (nanIssueExpire.sum()) 

nanIssueJob = permit3c['IssueYear'].isnull() & permit3c['JobYear'].isnull()
print ('No. of data records that have NaN issue and job Start date') 
print (nanIssueJob.sum()) 

nanExpireJob = permit3c['ExpireYear'].isnull() & permit3c['JobYear'].isnull()
print ('No. of data records that have NaN job start and expire date') 
print (nanExpireJob.sum()) 

No. of data records that have NaN for Job start date
3
No. of data records that have NaN for Issue date
2819
No. of data records that have NaN for Expire date
2161
No. of data records that have NaN for issue, expire and job start date
0
No. of data records that have NaN issue and expire date
2161
No. of data records that have NaN issue and job Start date
0
No. of data records that have NaN job start and expire date
0


### Now, the data has been cleaned to reflect Expire year from 2018 and Job Start from 2018

Based on the fact that there is NO data records that have NaN values for Issue & Job Start Date OR job start & expire date, as well as the fact that the 2 data records with NaN Job Start date has been checked to occur in 2018, it can be concluded that all data records has been checked and filtered to be open cases **from 2018**.

.

In [40]:
permit3c.columns

Index(['BOROUGH', 'CENSUS_TRACT', 'NTA_NAME', 'Zip Code', 'Block', 'Lot',
       'LATITUDE', 'LONGITUDE', 'Job Type', 'Work Type', 'Permit Type',
       'Permit Subtype', 'Oil Gas', 'Site Fill', 'Issuance Date',
       'Expiration Date', 'Job Start Date', 'JobStartDate', 'ExpireDate',
       'IssueDate', 'JobYear', 'ExpireYear', 'IssueYear'],
      dtype='object')

### However, still cannot claim that this is truly only 2018 open cases only

In [41]:
IssueExpire2019 = (permit3c['IssueYear']>2018) & (permit3c['ExpireYear']>2018)
IssueExpire2019.sum()

76492

In [42]:
IssueExpireJob2019 = (permit3c['IssueYear']>2018) & (permit3c['ExpireYear']>2018) & (permit3c['JobYear']>2018)
IssueExpireJob2019.sum()

60940

In [43]:
Issue2019Job2018 = (permit3c['IssueYear']>2018) & (permit3c['JobYear']==2018)
Issue2019Job2018.sum()

15551

### Need to drop the case where all three years are beyond 2018

and get data record number of 123774

In [44]:
len(permit3c) - IssueExpireJob2019.sum()

135228

In [45]:
permit4 = permit3c.drop(permit3c[(permit3c['IssueYear']>2018)\
                                  & (permit3c['ExpireYear']>2018)\
                                  & (permit3c['JobYear']>2018)].index)

In [46]:
len(permit4)

135228

In [47]:
permit4.head()

,BOROUGH,CENSUS_TRACT,NTA_NAME,Zip Code,Block,Lot,LATITUDE,LONGITUDE,Job Type,Work Type,...,Site Fill,Issuance Date,Expiration Date,Job Start Date,JobStartDate,ExpireDate,IssueDate,JobYear,ExpireYear,IssueYear
9,BROOKLYN,1134.0,East New York (Pennsylvania Ave),11207.0,3750,20,40.669502,-73.899127,NB,PL,...,NaN,07/20/2019 12:00:00 AM,07/19/2020 12:00:00 AM,07/19/2018 12:00:00 AM,2018-07-19,2020-07-19,2019-07-20,2018.0,2020.0,2019.0
11,BROOKLYN,1134.0,East New York (Pennsylvania Ave),11207.0,3750,21,40.669425,-73.899109,NB,PL,...,NaN,07/20/2019 12:00:00 AM,07/19/2020 12:00:00 AM,07/19/2018 12:00:00 AM,2018-07-19,2020-07-19,2019-07-20,2018.0,2020.0,2019.0
16,BRONX,71.0,Melrose South-Mott Haven North,10455.0,2294,32,40.815937,-73.916354,NB,NaN,...,ON-SITE,07/19/2019 12:00:00 AM,07/18/2020 12:00:00 AM,01/12/2018 12:00:00 AM,2018-01-12,2020-07-18,2019-07-19,2018.0,2020.0,2019.0
17,BROOKLYN,224.0,Borough Park,11219.0,5603,1,40.641170,-73.993464,A3,EQ,...,NaN,07/19/2019 12:00:00 AM,07/18/2020 12:00:00 AM,06/26/2018 12:00:00 AM,2018-06-26,2020-07-18,2019-07-19,2018.0,2020.0,2019.0
26,MANHATTAN,205.0,Morningside Heights,10025.0,1896,1,40.807413,-73.966265,A2,PL,...,NOT APPLICABLE,07/19/2019 12:00:00 AM,07/18/2020 12:00:00 AM,06/20/2018 12:00:00 AM,2018-06-20,2020-07-18,2019-07-19,2018.0,2020.0,2019.0


In [48]:
IssueNan = permit4['IssueYear'].isnull()
print ('No of data records where Issue year is NaN')
print (IssueNan.sum())

ExpireNan = permit4['ExpireYear'].isnull()
print ('No of data records where Expire year is NaN')
print (ExpireNan.sum())

JobNan = permit4['JobYear'].isnull()
print ('No of data records where Job year is NaN')
print (JobNan.sum())

No of data records where Issue year is NaN
2819
No of data records where Expire year is NaN
2161
No of data records where Job year is NaN
3


In [49]:
IssueNaNJob2019 = (permit4['IssueYear'].isnull()) & (permit4['JobYear']!=2018)

In [52]:
permit4b = permit4.drop(permit4[(permit4['IssueYear'].isnull())\
                                  & (permit4['JobYear']!=2018)].index)

In [54]:
len(permit4)- len(permit4b)

1332

In [56]:
len(permit4b)

133896

In [58]:
IssueNan = permit4b['IssueYear'].isnull()
print ('No of data records where Issue year is NaN')
print (IssueNan.sum())

ExpireNan = permit4b['ExpireYear'].isnull()
print ('No of data records where Expire year is NaN')
print (ExpireNan.sum())

JobNan = permit4b['JobYear'].isnull()
print ('No of data records where Job year is NaN')
print (JobNan.sum())

JobNot2018 = permit4b['JobYear']!=2018
print ('No of data records where Job year is not 2018')
print (JobNot2018.sum())

ExpireNanJob2019 = (permit4b['ExpireYear'].isnull()) & (permit4b['JobYear']!=2018)
print ('No of data records where Expire year is NaN and Job year is not 2018')
print(ExpireNanJob2019.sum())

IssueNanJob2019 = (permit4b['IssueYear'].isnull()) & (permit4b['JobYear']!=2018)
print ('No of data records where Issue year is NaN and Job year is not 2018')
print (IssueNanJob2019.sum())

No of data records where Issue year is NaN
1487
No of data records where Expire year is NaN
1117
No of data records where Job year is NaN
3
No of data records where Job year is not 2018
133
No of data records where Expire year is NaN and Job year is not 2018
0
No of data records where Issue year is NaN and Job year is not 2018
0


In [59]:
permit4b[(permit4b['JobYear'].isnull())]

,BOROUGH,CENSUS_TRACT,NTA_NAME,Zip Code,Block,Lot,LATITUDE,LONGITUDE,Job Type,Work Type,...,Site Fill,Issuance Date,Expiration Date,Job Start Date,JobStartDate,ExpireDate,IssueDate,JobYear,ExpireYear,IssueYear
37416,MANHATTAN,6.0,Lower East Side,10002.0,284,19,40.714143,-73.989128,NB,NaN,...,OFF-SITE,05/01/2019 12:00:00 AM,04/30/2020 12:00:00 AM,NaN,NaT,2020-04-30,2019-05-01,NaN,2020.0,2019.0
258696,MANHATTAN,6.0,Lower East Side,10002.0,284,19,40.714143,-73.989128,NB,NaN,...,OFF-SITE,01/12/2018 12:00:00 AM,02/23/2018 12:00:00 AM,NaN,NaT,2018-02-23,2018-01-12,NaN,2018.0,2018.0
414988,MANHATTAN,6.0,Lower East Side,10002.0,284,19,40.714143,-73.989128,NB,NaN,...,OFF-SITE,03/28/2017 12:00:00 AM,02/23/2018 12:00:00 AM,NaN,NaT,2018-02-23,2017-03-28,NaN,2018.0,2017.0


In [61]:
permit5 = permit4b.reset_index()

In [62]:
len(permit5)

133896

In [63]:
permit5

,index,BOROUGH,CENSUS_TRACT,NTA_NAME,Zip Code,Block,Lot,LATITUDE,LONGITUDE,Job Type,...,Site Fill,Issuance Date,Expiration Date,Job Start Date,JobStartDate,ExpireDate,IssueDate,JobYear,ExpireYear,IssueYear
0,9,BROOKLYN,1134.0,East New York (Pennsylvania Ave),11207.0,3750,20,40.669502,-73.899127,NB,...,NaN,07/20/2019 12:00:00 AM,07/19/2020 12:00:00 AM,07/19/2018 12:00:00 AM,2018-07-19,2020-07-19,2019-07-20,2018.0,2020.0,2019.0
1,11,BROOKLYN,1134.0,East New York (Pennsylvania Ave),11207.0,3750,21,40.669425,-73.899109,NB,...,NaN,07/20/2019 12:00:00 AM,07/19/2020 12:00:00 AM,07/19/2018 12:00:00 AM,2018-07-19,2020-07-19,2019-07-20,2018.0,2020.0,2019.0
2,16,BRONX,71.0,Melrose South-Mott Haven North,10455.0,2294,32,40.815937,-73.916354,NB,...,ON-SITE,07/19/2019 12:00:00 AM,07/18/2020 12:00:00 AM,01/12/2018 12:00:00 AM,2018-01-12,2020-07-18,2019-07-19,2018.0,2020.0,2019.0
3,17,BROOKLYN,224.0,Borough Park,11219.0,5603,1,40.641170,-73.993464,A3,...,NaN,07/19/2019 12:00:00 AM,07/18/2020 12:00:00 AM,06/26/2018 12:00:00 AM,2018-06-26,2020-07-18,2019-07-19,2018.0,2020.0,2019.0
4,26,MANHATTAN,205.0,Morningside Heights,10025.0,1896,1,40.807413,-73.966265,A2,...,NOT APPLICABLE,07/19/2019 12:00:00 AM,07/18/2020 12:00:00 AM,06/20/2018 12:00:00 AM,2018-06-20,2020-07-18,2019-07-19,2018.0,2020.0,2019.0
5,49,QUEENS,849.0,Flushing,11355.0,5062,14,40.755486,-73.834380,NB,...,NaN,07/19/2019 12:00:00 AM,07/18/2020 12:00:00 AM,07/31/2018 12:00:00 AM,2018-07-31,2020-07-18,2019-07-19,2018.0,2020.0,2019.0
6,68,MANHATTAN,183.0,Upper West Side,10025.0,1887,48,40.796049,-73.972978,A1,...,NOT APPLICABLE,07/19/2019 12:00:00 AM,07/18/2020 12:00:00 AM,07/20/2018 12:00:00 AM,2018-07-20,2020-07-18,2019-07-19,2018.0,2020.0,2019.0
7,86,BROOKLYN,505.0,East Williamsburg,11206.0,3052,23,40.707638,-73.943264,A2,...,NOT APPLICABLE,07/19/2019 12:00:00 AM,01/04/2020 12:00:00 AM,07/09/2018 12:00:00 AM,2018-07-09,2020-01-04,2019-07-19,2018.0,2020.0,2019.0
8,145,MANHATTAN,201.0,Lower East Side,10002.0,269,41,40.713684,-73.986336,A3,...,NaN,07/19/2019 12:00:00 AM,07/18/2020 12:00:00 AM,07/18/2018 12:00:00 AM,2018-07-18,2020-07-18,2019-07-19,2018.0,2020.0,2019.0
9,151,BROOKLYN,379.0,Stuyvesant Heights,11233.0,1514,11,40.681614,-73.918834,NB,...,USE UNDER 300 CU.YD,07/19/2019 12:00:00 AM,11/12/2019 12:00:00 AM,03/29/2018 12:00:00 AM,2018-03-29,2019-11-12,2019-07-19,2018.0,2019.0,2019.0


In [64]:
permit5.to_csv('permit.csv')